## Tabela Silver para Dimenssão de Clima

#### Tratamento do Dataframe para obter informações desejadas

Aqui vamos filtrar apenas as colunas desejadas e realizar o tratamentod e tipagem e limpeza de dados necessárias

Vamos realizar alguns tratamentos

    Seleção de colunas necessárias
    Configurar colunas para o padrão
    Filtrando dados indesejados
    Alteração de tipagem de dados
    Removendo dados nulos
    Salvando arquivo de forma compactada

In [0]:
# Importando Bibliotecas para uso do Spark
from pyspark.sql import SparkSession # Import para inciar o spark
from pyspark.sql.types import * # Todos as configurações do SQL de tipagem no spark
from pyspark.sql.functions import * # Todas as funções de SQL do Spark
from pyspark.sql.window import Window

In [0]:
# Diretorio Bronze, com os dados brutos das causas dos acidentes
Bronze_Acidentes_Causas = '/Volumes/workspace/dw_acidentes/01_bronze/Acidentes_Causas/'

# Local para armazenar dimenssão
Diretorio_Silver = '/Volumes/workspace/dw_acidentes/02_silver'


In [0]:
# Lê os arquivos CSV descompactado
df_Acidentes_Causas = spark.read.parquet(Bronze_Acidentes_Causas)

In [0]:
# Selecionado apenas as colunas necessária para criar o modelo Dimenssional de Clima
df_clima = df_Acidentes_Causas.select(['fase_dia', 'condicao_metereologica'])

In [0]:
# Removendo Duplicadas para criar o campo chave
df_clima = df_clima.dropDuplicates() 

In [0]:
# Criando a função de janela e ordenando todos as colunas aplicando linha a linha 
window = Window.orderBy('fase_dia', 'condicao_metereologica')

# Aplicando a função row_number e over para cada linha incluir o ID incremental.
df_clima = df_clima.withColumn(
    "id_clima",
    row_number().over(window)
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Definido a ordem conforme a documentação desejada
df_clima = df_clima.select(['id_clima', 'fase_dia', 'condicao_metereologica'])

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Validando os campos para realizar os tratamentos
df_clima.printSchema()

root
 |-- id_clima: integer (nullable = false)
 |-- fase_dia: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)



In [0]:
# Visualizando o primeiros registros para visualizar o resultado final 
display(df_clima.limit(5))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


id_clima,fase_dia,condicao_metereologica
1,Amanhecer,Chuva
2,Amanhecer,Céu Claro
3,Amanhecer,Garoa/Chuvisco
4,Amanhecer,Granizo
5,Amanhecer,Ignorado


In [0]:
# Salvando o arquivo em parquet com compressão gzip 
df_clima.write.parquet(
    f'{Diretorio_Silver}/Dim_Clima', 
    mode="overwrite",         # Aqui defino para sempre sobrescrever o arquivo
    compression="gzip"        # Comando para salvar o arquivo com compactação
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
